In [1]:
import numpy as np
import pandas as pd
import re
import uuid
import requests

import matplotlib.pyplot as plt
import matplotlib as mpl

import seaborn as sns
sns.set()

%matplotlib inline


***
# Initialize and perform base level cleaning #

In [87]:
cleaned_csv_location = '../cleaned_tweets.csv'
def save_cleaned_df(df):
    df.to_csv(cleaned_csv_location)
    
def read_cleaned_df():    
    return pd.read_csv(cleaned_csv_location, index_col='id')

In [88]:
#tweet_id needs to be an object because pandas tries to convert it to scientific notation
russia_df = pd.read_csv('../tweets.csv', dtype={'tweet_id': np.object})

russia_df['created_datetime'] = pd.to_datetime(russia_df['created_str'])
# converts from UTC to EST
russia_df['created_datetime'] = russia_df['created_datetime'] - pd.Timedelta(hours=5)
# Make the created_datetime the index
#russia_df.set_index('created_datetime', inplace=True)

# Delete columns we wont use
del russia_df['created_str']
del russia_df['created_at']
del russia_df['posted']
# I parse these out myself later in this notebook
del russia_df['hashtags']
del russia_df['mentions']

# There are 21 null text tweets. Delete for now, but amy come back and look later
russia_df = russia_df[~russia_df['text'].isnull()]

russia_df['id'] = [uuid.uuid4() for _ in range(0,russia_df.shape[0])]
russia_df.set_index('id',inplace=True)
save_cleaned_df(russia_df)

russia_df.sample(5)


,user_id,user_key,retweet_count,retweeted,favorite_count,text,tweet_id,source,expanded_urls,retweeted_status_id,in_reply_to_status_id,created_datetime
id,,,,,,,,,,,,
4c40e3c1-3945-488f-a5a5-7c79ec15417d,2.541946e+09,mr_clampin,NaN,NaN,NaN,RT @Freddiemaize: I have a huge crush on @Robe...,809032410197204994,NaN,[],NaN,NaN,2016-12-14 08:48:52
d9aa201c-e2ba-4636-8c02-03d0a514bafd,1.727482e+09,melvinsroberts,NaN,NaN,NaN,RT @OnPirateSat: #thanks @Nik_No_C @PunkSwapMe...,781120912502370304,NaN,[],NaN,NaN,2016-09-28 08:18:32
0bb74709-396d-405e-b4ab-65691527ee0d,1.656024e+09,imissobama,0.0,False,0.0,RT @Toure: Trump will not be the last of the b...,790008286015815680,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],7.900038e+17,NaN,2016-10-22 20:53:47
7e276adf-23dd-4ead-9122-9b9b37f5f3ba,1.623180e+09,jeffreykahunas,NaN,NaN,NaN,RT @greta: Powell? he is the one who told the ...,764145521955000320,NaN,[],NaN,NaN,2016-08-12 12:04:24
d99a2b64-9b56-495f-ad57-c2ddab4fe2a6,2.532612e+09,kathiemrr,NaN,NaN,NaN,RT @AnnoGalactic: #FamousCreatures\n\nBull McC...,785413943225233408,NaN,[],NaN,NaN,2016-10-10 04:37:31


***
# Extract hashtags #

In [89]:
tweets_with_hashtags = russia_df[russia_df['text'].str.contains('#\w+')]

print(tweets_with_hashtags.shape[0])
tweets_with_hashtags.sample(2)

100651


,user_id,user_key,retweet_count,retweeted,favorite_count,text,tweet_id,source,expanded_urls,retweeted_status_id,in_reply_to_status_id,created_datetime
id,,,,,,,,,,,,
a044f436-1fe6-44a0-bdf7-8b07f59f5545,1.658203e+09,laurabaeley,NaN,NaN,NaN,We must protect ourselves by whatever means ne...,672819221664018432,NaN,[],NaN,NaN,2015-12-04 11:46:17
2bd1474a-ebef-4985-b402-283566de0f20,1.623180e+09,jeffreykahunas,NaN,NaN,NaN,Complicated - Complicated - Avril Lavigne http...,737557603455139844,NaN,[],NaN,NaN,2016-05-31 03:13:30


In [90]:
hash_df = russia_df['text'].str.extractall('(#\w+)')

In [91]:
hash_df = hash_df.reset_index().set_index('id')
del hash_df['match']

hash_df.to_csv('../csvs/hashtags.csv')

In [92]:
hash_df[0].value_counts().head(10)

#politics             3638
#tcot                 2839
#MAGA                 2538
#PJNET                2147
#news                 2046
#Trump                1851
#Merkelmussbleiben    1108
#TrumpForPresident    1088
#WakeUpAmerica        1061
#NeverHillary          976
Name: 0, dtype: int64

***
# Extract mentions #

In [93]:
mentions_df = russia_df['text'].str.extractall('(@\w+)')
mentions_df = mentions_df.reset_index().set_index('id')
del mentions_df['match']

mentions_df.to_csv('../csvs/mentions.csv')

In [94]:
mentions_df[0].value_counts().head(10)

@realDonaldTrump    4567
@midnight           2584
@HillaryClinton     2343
@blicqer            2238
@Conservatexian     1105
@POTUS               909
@FoxNews             744
@YouTube             557
@PrisonPlanet        554
@nine_oh             538
Name: 0, dtype: int64

***
# source #

** 'source' is the client that performed the tweet (not consistently stamped) **

In [95]:
russia_df.sort_values('source', ascending=False)['source'].value_counts()[:5]

<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                     42685
<a href="http://twitterfeed.com" rel="nofollow">twitterfeed</a>                         6926
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>     6409
<a href="http://twibble.io" rel="nofollow">Twibble.io</a>                               1491
<a href="http://dlvr.it" rel="nofollow">dlvr.it</a>                                      243
Name: source, dtype: int64

***
# Extract URLs #
This doesn't get all of the urls, but it gets most of them. Some are truncated (seems to be the case for RTs)

In [96]:
url_regex = '(https?:\/\/t.co\/[a-zA-Z0-9\-]+)'

In [97]:

url_df = russia_df['text'].str.extractall(url_regex)
url_df = url_df.reset_index().set_index('id')
del url_df['match']

url_df.to_csv('../csvs/urls.csv')

In [98]:
url_df[0].value_counts().head(10)

https://t.co/eMX9JgTv3v    269
https://t.co/aXQaNDGlF7    163
https://t.co/iChLzWqg0Y    146
https://t.co/rRZgBcCxBO    145
https://t.co/XDBaPnHm6c    127
https://t.co/telXZBWPRi    123
https://t.co/rRZgBcU8tm    114
https://t.co/yX3iMOFIOR    104
https://t.co/1KPXto2HfW     99
https://t.co/cdnQqWB38H     98
Name: 0, dtype: int64

In [99]:
#unwrapped_urls = url_df[0].apply(lambda x: requests.get(x).url)

## Remove URLs from the text and insert the result in a special 'processed_text' column ##
This column will be used for clustering

In [100]:
russia_df['processed_text'] = russia_df['text'].str.replace(url_regex, '')

save_cleaned_df(russia_df)

# Let's classify some tweets that we know of#

### @midnight ###

In [101]:
russia_df['@midnight'] = russia_df['text'].str.contains('@midnight', flags=re.IGNORECASE)

save_cleaned_df(russia_df)

### #merkelmussbleiben ###

In [102]:
russia_df['merkel'] = russia_df['text'].str.contains('#merkelmussbleiben', flags=re.IGNORECASE)
save_cleaned_df(russia_df)

## Retweets ##
** Assign a is_rt column and strip the RT symbol from the tweets **

In [103]:
russia_df['is_rt'] = russia_df['text'].str.contains('\\bRT\\b', na=False)

russia_df['processed_text'] = russia_df['processed_text'].str.replace('\\bRT\\b', '')
save_cleaned_df(russia_df)

## Misc Cleaning ##

** Remove incomplete https urls **

Many RTs are truncated at the end. When the end contains a link, it often causes the earlier 'URL' regex to miss stripping the text

In [104]:
partial_url_regex = '\\bhttp\S*$'
russia_df['processed_text'] = russia_df['processed_text'].str.replace(partial_url_regex, '')
save_cleaned_df(russia_df)

** Strip ampersands that exist by themselves **

In [105]:
russia_df['processed_text'] = russia_df['processed_text'].str.replace('&amp;', '&')
russia_df['processed_text'] = russia_df['processed_text'].str.replace('&amp;', '&')
save_cleaned_df(russia_df)